# APA Polynomial regression 

In [ ]:
# Uncomment to upgrade packages
# !pip3 install pandas --upgrade --user --quiet
# !pip3 install numpy --upgrade --user --quiet 
# !pip3 install scipy --upgrade --user --quiet
# !pip3 install statsmodels --upgrade --user --quiet 
# !pip3 install scikit-learn --upgrade --user --quiet
%load_ext autoreload

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('precision', 3)

In [ ]:
# extra imports
from numpy.random import uniform, normal
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

Fix the random number generator so the experiment is reproducible

In [ ]:
np.random.seed(7)

We will approximate the function $cos(3\pi x)$ using polynomials

In [ ]:
def feval(x):
    return np.cos(3*np.pi*x)

def fgen(N, sigma):
    x = np.sort(uniform( a,b,N))
    t = feval(x) + normal(loc=0, 
                               scale=sigma,
                               size=N)  
    return x,t



We start generating a sample of 30 examples generating the data assuming that has gaussian noise $N(0,0.25^2)$

In [ ]:
N = 30
a = 0
b = 1
sigma = 0.25

In [ ]:
x,t = fgen(N, sigma)
sample = pd.DataFrame({'input':x,'target':t})

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(sample.input, sample.target, 'o')
ax.plot(np.linspace(0, 1,num=30), 
        feval(np.linspace(0, 1,num=30)));

We generate also a vaidation set to evaluate the error with unseen data

In [ ]:
N_valid = 1000
x_valid, t_valid = fgen(N_valid, sigma)
valid_sample =  pd.DataFrame({'input':x_valid,'target':t_valid})

We fit polynomials for a range of degrees

We sample the range (0,1) and plot the predictions

In [ ]:
p = 1
q = 26

coef = []
model = []
norm_mse_train = []
norm_mse_valid = []

for i in range(p,q):
    cmodel = LinearRegression()
    cmodel.fit(np.vander(sample.input,i+1, increasing=True), sample.target)
    coef.append(cmodel.coef_)
    predictions = cmodel.predict(np.vander(sample.input,
                                              i+1, increasing=True))  
   
    
    norm_mse_train.append(mean_squared_error(predictions, sample.target))
    
    pred_val = cmodel.predict(np.vander(valid_sample.input,
                                              i+1, increasing=True))  
    norm_mse_valid.append(mean_squared_error(pred_val, valid_sample.target))
    model.append(cmodel)
0;

In [ ]:
fig = plt.figure(figsize=(16,7))


for f, i in enumerate([1,2,3,8,15,25]):
    ax = fig.add_subplot(2,3,f+1)
    ax.plot(sample.input, sample.target, '+')
    ax.plot(np.linspace(0, 1,num=40), 
            feval(np.linspace(0, 1,num=40)), '.');
    ax.plot(np.linspace(0, 1,num=40), 
            model[i-1].predict(np.vander(np.linspace(0, 1,num=40),
                                              i+1, 
                                              increasing=True)));
    plt.ylim(-1.2,1.5)
    plt.title('Degree %d'%i)
0;


This is the difference among the train and validation data

In [ ]:
pol =15
fig, ax = plt.subplots(figsize=(8,6))
plt.plot(range(1,pol+1), norm_mse_train[0:pol], '-+', label='Error.TR')
plt.plot(range(1,pol+1), norm_mse_valid[0:pol], '-o', label='Error.VA')
plt.legend();

These are the means of the abssolute value of the coefficients fitted for each polynomial in logarithmic scale

In [ ]:
pol=25
means = []
for m in model:
    means.append(np.mean(np.abs(m.coef_)))
fig, ax = plt.subplots(figsize=(8,6))
plt.plot(range(1,pol+1), means[0:pol], '-+', label='Coefficients means')
plt.yscale('log')
plt.legend();


Now we repeat the same but using regularization, we first find the best regularization parameter using cross validation

In [ ]:
p = 1
q = 26

coef = []
model = []
norm_mse_train = []
norm_mse_valid = []


rlambda = np.linspace(0.001, 0.5, num=50)
for i in range(p,q):
    cmodel = RidgeCV(alphas=rlambda, scoring=make_scorer(mean_squared_error, greater_is_better=False))
    cmodel.fit(np.vander(sample.input,i+1, increasing=True), sample.target)
    coef.append(cmodel.coef_)
    predictions = cmodel.predict(np.vander(sample.input,
                                              i+1, increasing=True))  
   
    
    norm_mse_train.append(mean_squared_error(predictions, sample.target))
    
    pred_val = cmodel.predict(np.vander(valid_sample.input,
                                              i+1, increasing=True))  
    norm_mse_valid.append(mean_squared_error(pred_val, valid_sample.target))
    model.append(cmodel)
0;

In [ ]:
fig = plt.figure(figsize=(14,7))


for f, i in enumerate([1,2,3,8,15,25]):
    ax = fig.add_subplot(2,3,f+1)
    ax.plot(sample.input, sample.target, '+')
    ax.plot(np.linspace(0, 1,num=40), 
            feval(np.linspace(0, 1,num=40)), '.');
    ax.plot(np.linspace(0, 1,num=40), 
            model[i-1].predict(np.vander(np.linspace(0, 1,num=40),
                                              i+1, 
                                              increasing=True)));
    plt.ylim(-1.2,1.5)
    plt.title('Degree %d'%i)
0;


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
pol =25
plt.plot(range(1,pol+1), norm_mse_train[0:pol], '-+', label='Error.TR')
plt.plot(range(1,pol+1), norm_mse_valid[0:pol], '-o', label='Error.VA')
plt.legend();

Now the coefficients are smaller due to the regularization, now we do not need logarithmic scale to represent them

In [ ]:
means = []
for m in model:
    means.append(np.mean(np.abs(m.coef_)))
fig, ax = plt.subplots(figsize=(8,6))
plt.plot(range(1,pol+1), means[0:pol], '-+', label='Coefficients means')
plt.legend();